In [1]:
# test_augmentation.py

import os
import torch
import scipy.io.wavfile as wav

from src.augmentation_func import RandomStartCrop, RandomStartCrop, RandomSpeedChange  # <-- jetzt aus eigener Datei

# === Konfiguration für den Test ===
AUDIO_FILE = r"../music/blues/blues.00000.wav"  # Pfad anpassen!
SAMPLING_RATE_FOR_TEST = 22050                 # an deine Config anpassen
DURATION_SECONDS = 15
# z.B. 15 Sekunden

def load_audio_as_tensor(file_path: str) -> tuple[int, torch.Tensor]:
    """Liest eine WAV-Datei ein und gibt (sampling_rate, audio_tensor) zurück."""
    sr, data = wav.read(file_path)
    # In Float32 umwandeln
    audio_tensor = torch.from_numpy(data).to(torch.float32)

    # Falls Stereo: auf Mono mitteln
    if audio_tensor.dim() > 1:
        audio_tensor = audio_tensor.mean(dim=1)

    return sr, audio_tensor

def save_tensor_as_wav(file_path: str, sr: int, audio_tensor: torch.Tensor):
    """Speichert einen Float32-Tensor wieder als WAV (int16)."""
    max_val = audio_tensor.abs().max()
    if max_val > 0:
        audio_norm = audio_tensor / max_val
    else:
        audio_norm = audio_tensor

    audio_int16 = (audio_norm * 32767).clamp(-32768, 32767).to(torch.int16).numpy()
    wav.write(file_path, sr, audio_int16)

def main():
    # 1) Datei laden
    if not os.path.exists(AUDIO_FILE):
        raise FileNotFoundError(f"Audio file not found: {AUDIO_FILE}")

    sr, audio = load_audio_as_tensor(AUDIO_FILE)
    print(f"Original sampling rate: {sr} Hz")
    print(f"Original length: {audio.shape[0]} samples (~{audio.shape[0] / sr:.2f} s)")

    if sr != SAMPLING_RATE_FOR_TEST:
        print(f"⚠️ Warnung: Datei hat SR={sr}, Test-SR={SAMPLING_RATE_FOR_TEST}")

    # 2) Augmentation-Objekt anlegen
    augmenter = RandomStartCrop(sampling_rate=sr, duration=DURATION_SECONDS)

     # 3) Mehrere zufällige Ausschnitte erzeugen
    num_examples = 3
    for i in range(num_examples):
        augmented = augmenter(audio)

        start_sample = augmenter.last_start
        end_sample = augmenter.last_end
        start_sec = start_sample / sr if start_sample is not None else None
        end_sec = end_sample / sr if end_sample is not None else None

        print(f"\nBeispiel {i+1}:")
        print(f"  Startpunkt: sample {start_sample} (~{start_sec:.2f} s)")
        print(f"  Endpunkt:   sample {end_sample} (~{end_sec:.2f} s)")
        print(f"  Augmentierte Länge: {augmented.shape[0]} samples (~{augmented.shape[0] / sr:.2f} s)")

        out_path = f"augmented_example_{i+1}.wav"
        save_tensor_as_wav(out_path, sr, augmented)
        print(f"  Gespeichert als: {out_path}")
        
        augment_speed = RandomSpeedChange(factors=(0.7, 1.3))


    for i in range(num_examples):
        # erst Speed-Änderung, dann RandomStartCrop
        stretched = augment_speed(audio)
        cropped = augmenter(stretched)

        print(f"\nBeispiel {i+1}:")
        print(f"  Speed-Faktor: {augment_speed.last_factor}")
        print(f"  Start: {augmenter.last_start}, Ende: {augmenter.last_end}")
        # ...



if __name__ == "__main__":
    main()


Original sampling rate: 22050 Hz
Original length: 661794 samples (~30.01 s)

Beispiel 1:
  Startpunkt: sample 160116 (~7.26 s)
  Endpunkt:   sample 490866 (~22.26 s)
  Augmentierte Länge: 330750 samples (~15.00 s)
  Gespeichert als: augmented_example_1.wav

Beispiel 2:
  Startpunkt: sample 292490 (~13.26 s)
  Endpunkt:   sample 623240 (~28.26 s)
  Augmentierte Länge: 330750 samples (~15.00 s)
  Gespeichert als: augmented_example_2.wav

Beispiel 3:
  Startpunkt: sample 63572 (~2.88 s)
  Endpunkt:   sample 394322 (~17.88 s)
  Augmentierte Länge: 330750 samples (~15.00 s)
  Gespeichert als: augmented_example_3.wav


NameError: name 'num_examples' is not defined